In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
import numpy as np
import os
import matplotlib.pyplot as plt

from src.datasets import BiosensorDataset
from src.unet import UNet
from src.train import train_model

In [2]:
BIO_LENGTH = 32
MASK_SIZE = 80
BATCH_SIZE = 8
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device {device}')

Using device cuda


In [3]:
from src.train_lightning import UNetLightningModule, BiosensorDataModule
from src.losses import DiceLoss, IoULoss

from argparse import Namespace
from pathlib import Path
import torch
from pytorch_lightning import Trainer
from torch.nn import BCEWithLogitsLoss

# Define the hyperparameters
args = Namespace(
    lr=0.001,
    epochs=10,
    batch_size=BATCH_SIZE,
    amp=False,
    bilinear=False,
    data_path='data_with_centers/'
)

# Define the loss function
criterion = DiceLoss()

# Initialize the model and data module
model = UNetLightningModule(learning_rate=args.lr, channels=BIO_LENGTH, classes=1, loss_func=criterion, amp=args.amp, bilinear=args.bilinear)
data_module = BiosensorDataModule(data_path=args.data_path, batch_size=args.batch_size, biosensor_length=BIO_LENGTH, mask_size=MASK_SIZE)

# Initialize the trainer
trainer = Trainer(max_epochs=args.epochs, accelerator='gpu' if torch.cuda.is_available() else 'cpu', precision=16 if args.amp else 32)

# Train the model
trainer.fit(model, data_module)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\wittd\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA GeForce RTX 3070') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precisi

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\wittd\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
c:\Users\wittd\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


torch.Size([8, 1, 80, 80]) torch.Size([8, 80, 80])


AssertionError: 

In [ ]:
data_path = 'data_with_centers/'

transform = transforms.Compose([
    # transforms.Normalize((0.5,), (0.5,))
])

dataset = BiosensorDataset(data_path, mask_type=bool, biosensor_length=BIO_LENGTH, mask_size=80)

TRAIN_SIZE = int(len(dataset)*0.86)
VAL_SIZE = len(dataset) - TRAIN_SIZE

train_data, val_data = torch.utils.data.random_split(dataset, [TRAIN_SIZE, VAL_SIZE])
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True)

model = UNet(n_channels=BIO_LENGTH, n_classes=1)
model = model.to(device)

try:
    train_model(
        model,
        device,
        train_loader,
        val_loader,
        learning_rate=0.01,
        epochs=10,
        # checkpoint_dir=checkpoint_dir,
        amp=True,
    )
except torch.cuda.OutOfMemoryError:
    torch.cuda.empty_cache()
    print('Detected OutOfMemoryError!')